In [2]:
import pandas as pd
import numpy as np

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [5]:
data = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")
data.head(2)

,Unnamed: 0,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air
0,0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...
1,2,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...


In [10]:
metadata = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/Finalized_dataset_1805_exams_with_Age.xlsx")
print(metadata.columns)
metadata.head(2)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0_x', 'directory',
       'source_directory', 'patient_directory', 'PET-CT_info', 'system', 'npr',
       'scan_date', 'Unnamed: 0_y', 'dicom_img', 'patient_id', 'patient_age',
       'patient_weight', 'patient_size', 'imgsz_x', 'imgsz_y', 'num_slices',
       'voxsz_x', 'voxsz_y', 'slice_thickness', 'image_size', 'voxel_size',
       'img_pos', 'img_orient', 'for_uid', 'att_corr', 'recons_method',
       'image_type', 'aquisition_dt', 'aquisition_time', 'study_desc',
       'series_desc', 'protocol', 'corr_img', 'modality', 'manufacturer',
       'manufacturer_model', 'radiopharmaceutical',
       'radiopharmaceutical_volume', 'radiopharmaceutical_start_time',
       'radionuclide_total_dose', 'radionuclide_half_life',
       'radionuclide_positron_fraction',
       'radiopharmaceutical_start_date_time'],
      dtype='object')


,Unnamed: 0.1,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_...,ASPTCTX0001,lpr385705046400,20140313,...,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D...,ASPTCTX0001,lpr385705046400,20140313,...,PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [11]:
metadata = metadata[["npr", "scan_date", "patient_age"]]
print(metadata.shape)
metadata.drop_duplicates(inplace=True)
print(metadata.shape)
metadata.head(2)

(3610, 3)
(1805, 3)


,npr,scan_date,patient_age
0,lpr385705046400,20140313,33
2,lpr415675513429,20190201,32


In [12]:
final = pd.merge(data, metadata, how="inner", left_on=["patient_ID", "scan_date"], right_on=["npr", "scan_date"], suffixes=["_l","_r"])
final.head(2)

,Unnamed: 0,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,npr,patient_age
0,0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,lpr385705046400,33
1,2,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,lpr415675513429,32


In [16]:
final_data = final.drop(columns=["Unnamed: 0", "npr"])

In [18]:
final_data.to_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")

In [1]:
testing = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/dataset_for_training_regression.xlsx")

NameError: name 'pd' is not defined

In [3]:
testing

,Unnamed: 0,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,patient_age
0,0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
1,1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,32
2,2,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
3,3,lpr884752331885,20181116,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,69
4,4,npr100169878450,20130412,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,1800,npr999470312453,20220201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,72
1801,1801,npr999561275820,20130305,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,82
1802,1802,npr999612862346,20210212,/media/andres/T7 Shield1/UCAN_project

In [14]:
sort_test = testing.sort_values(by="patient_ID")
final_test = sort_test.drop(columns="Unnamed: 0").reset_index(drop=True)
final_test

,patient_ID,scan_date,SUV_MIP,CT_MIP,SUV_bone,CT_bone,SUV_lean,CT_lean,SUV_adipose,CT_adipose,SUV_air,CT_air,patient_age
0,lpr385705046400,20140313,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
1,lpr415675513429,20190201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,32
2,lpr415675513429,20190320,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,33
3,lpr884752331885,20181116,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,69
4,npr100169878450,20130412,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,npr999470312453,20220201,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,72
1801,npr999561275820,20130305,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UCAN_project/collages...,82
1802,npr999612862346,20210407,/media/andres/T7 Shield1/UCAN_project/collages...,/media/andres/T7 Shield1/UC

In [16]:
train_data = final_test[:int(final_test.shape[0] * 0.7)].patient_ID.tolist()
train = final_test[final_test.patient_ID.isin(train_data)]
val = final_test[~final_test.patient_ID.isin(train_data)]

In [17]:
train.shape

(1263, 13)

In [18]:
val.shape

(542, 13)

In [22]:
train["patient_age"].nunique()

71

In [12]:
output_path = "/media/andres/T7 Shield1/UCAN_project/collages/"
df_for_collages = pd.read_excel("/media/andres/T7 Shield1/UCAN_project/df_for_collages.xlsx")
df_for_collages.shape

(3646, 14)

In [15]:
df_for_collages["patient_ID"].nunique()

1076

In [16]:
df_for_collages["scan_date"].nunique()

1168

### Updating file names for new system

##### regression

In [2]:
testing = pd.read_excel("/home/ashish/Ashish/UCAN/dataset_for_training_regression.xlsx")

In [3]:
display_full(testing.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                              SUV_MIP                                                                              CT_MIP                                                                              SUV_bone                                                                              CT_bone                                                                              SUV_lean                                                                              CT_lean                                                                              SUV_adipose                                                                              CT_adipose                                                                              SUV_air                                                                              CT_air  patient_age
0           0  lpr385705046400   20140313  /media/andres/T7 Shield1/UCAN_project/collage

In [8]:
testing = testing.replace('/media/andres/T7 Shield1/UCAN_project', '/home/ashish/Ashish/UCAN', regex=True)
display_full(testing.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air  patient_age
0           0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collage

In [9]:
testing.drop(columns='Unnamed: 0', inplace=True)
display_full(testing.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air  patient_age
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/201403

In [10]:
testing.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_regression_v1.xlsx", index=False)

##### classification

In [3]:
classification_data = pd.read_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification.xlsx")

In [4]:
display_full(classification_data.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                              SUV_MIP                                                                              CT_MIP                                                                              SUV_bone                                                                              CT_bone                                                                              SUV_lean                                                                              CT_lean                                                                              SUV_adipose                                                                              CT_adipose                                                                              SUV_air                                                                              CT_air
0           0  lpr385705046400   20140313  /media/andres/T7 Shield1/UCAN_project/collages/lpr38570504

In [5]:
classification_data = classification_data.replace('/media/andres/T7 Shield1/UCAN_project', '/home/ashish/Ashish/UCAN', regex=True)
display_full(classification_data.head(2))

   Unnamed: 0       patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air
0           0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr38570504

In [6]:
classification_data.drop(columns='Unnamed: 0', inplace=True)
display_full(classification_data.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/CT_MIP.npy  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_bone.n

In [12]:
linked_data = pd.read_excel("/home/ashish/Ashish/UCAN/linked_data_387matches_27nov2023.xlsx")
linked_data = linked_data[["npr", "sex", "diagnosisGroups", "Diagnosis", "Diagnosis 2", "TREATMENT", "indication"]].copy()
display_full(linked_data.head(2))

               npr      sex diagnosisGroups                             Diagnosis Diagnosis 2     TREATMENT indication
0  lpr385705046400  UNKNOWN            HEMA  C81.9 Hodgkins lymfom, ospecificerat         NaN  CHEMOTHERAPY   CURATIVE
1  lpr415675513429  UNKNOWN            HEMA  C81.9 Hodgkins lymfom, ospecificerat         NaN  CHEMOTHERAPY   CURATIVE


In [16]:
classification_data_v1 = pd.merge(classification_data, linked_data, how="inner", left_on=["patient_ID"], right_on=["npr"], suffixes=['','_'])
display_full(classification_data_v1.head(2))

        patient_ID  scan_date                                                                 SUV_MIP                                                                 CT_MIP                                                                 SUV_bone                                                                 CT_bone                                                                 SUV_lean                                                                 CT_lean                                                                 SUV_adipose                                                                 CT_adipose                                                                 SUV_air                                                                 CT_air              npr      sex diagnosisGroups                             Diagnosis Diagnosis 2     TREATMENT indication
0  lpr385705046400   20140313  /home/ashish/Ashish/UCAN/collages/lpr385705046400/20140313/SUV_MIP.npy  /home/ashish/Ashish/UCA

In [17]:
classification_data_v1.to_excel("/home/ashish/Ashish/UCAN/dataset_for_training_classification_v1.xlsx", index=False)